In [11]:
import pandas as pd
import numpy as np

In [12]:
flights = pd.read_csv('data/Filghts_TEC_Valid.csv')
sales = pd.read_csv('data/Sales_TEC_Valid.csv')

In [13]:
flights['STD'] = pd.to_datetime(flights['STD'])
flights['STA'] = pd.to_datetime(flights['STA'])

In [14]:
flights = flights.sort_values(by=['Flight_ID'])
flights.reset_index(drop=True, inplace=True)

In [15]:
import pandas as pd

def keep_last_occurrence(df, unique_columns):
    # Sort the DataFrame by 'Aeronave' column in descending order to keep the last occurrence
    df = df.sort_values(by='Aeronave', ascending=False)
    # Drop duplicates based on the specified unique columns, keeping the first occurrence
    df = df.drop_duplicates(subset=unique_columns, keep='first')
    return df




In [16]:
flights_copy = flights.copy()


# Define the unique columns (all columns except 'Aeronave')
unique_columns = [col for col in flights_copy.columns if col != 'Aeronave']

# Apply the function
result = keep_last_occurrence(flights_copy, unique_columns)


In [28]:
result.to_csv('data/result.csv', index=False)

In [17]:
import pandas as pd

def consolidate_flights(df):
    # Convert 'STD' and 'STA' to datetime
    df['STD'] = pd.to_datetime(df['STD'])
    df['STA'] = pd.to_datetime(df['STA'])

    # Group by 'Flight_ID' to consolidate information
    result = df.groupby('Flight_ID').apply(consolidate_group).reset_index(drop=True)

    # Classify remaining flights based on the flight duration
    result = classify_flight_type(result)

    return result

def consolidate_group(group):
    if len(group) > 1:  # Check if the group has more than one flight, meaning it's a merged flight
        tipo_vuelo = 'Largo'
    else:
        tipo_vuelo = 'Corto'  # Default to Corto, will be reclassified if necessary

    # Sort by 'STD' to get the order of the flights
    group = group.sort_values(by='STD')

    # Consolidate information
    consolidated = {
        'Flight_ID': group['Flight_ID'].iloc[0],
        'Aeronave': group['Aeronave'].iloc[0],
        'Capacity': group['Capacity'].iloc[0],
        'DepartureStation': group['DepartureStation'].iloc[0],
        'ArrivalStation': group['ArrivalStation'].iloc[-1],
        'Destination_Type': group['Destination_Type'].iloc[-1],
        'Origin_Type': group['Origin_Type'].iloc[0],
        'STD': group['STD'].iloc[0],
        'STA': group['STA'].iloc[-1],
        'Passengers': compute_average_or_single_value(group['Passengers']),
        'Bookings': compute_average_or_single_value(group['Bookings']),
        'tipo_vuelo': tipo_vuelo  # Assign tipo_vuelo based on consolidation
    }

    return pd.Series(consolidated)

def classify_flight_type(df):
    # Only reclassify flights that were initially labeled as 'Corto'
    corto_mask = df['tipo_vuelo'] == 'Corto'
    time_difference = (df.loc[corto_mask, 'STA'] - df.loc[corto_mask, 'STD']).dt.total_seconds() / 60

    df.loc[corto_mask, 'tipo_vuelo'] = time_difference.apply(lambda x: 'Largo' if x > 150 else 'Corto')
    return df

def compute_average_or_single_value(series):
    # Filter out NaN values
    non_nan_values = series.dropna()

    if len(non_nan_values) == 0:
        return np.nan
    elif len(non_nan_values) == 1:
        return non_nan_values.iloc[0]
    else:
        return non_nan_values.mean()

In [18]:
consolidated_df = consolidate_flights(result)


In [29]:
consolidated_df.to_csv('data/consolidated.csv', index=False)

In [36]:
# Convert 'STD' to datetime if not already
consolidated_df['STD'] = pd.to_datetime(consolidated_df['STD'])

# Filter data for the year 2023
data_2023 = consolidated_df[consolidated_df['STD'].dt.year == 2023]

# Filter data for the year 2024
data_2024 = consolidated_df[consolidated_df['STD'].dt.year == 2024]

In [52]:
data_2023.shape

(119645, 12)

In [53]:
def clean_passenger_col(df):
    # Select only records of flights with more than one passenger
    df = df[df['Passengers'] >= 70]
    # Replace the number of passengers with the capacity when the number of passengers is greater than capacity
    df['Passengers'] = df[['Passengers', 'Capacity']].min(axis=1)
    return df

In [54]:
data_2023_modelo = clean_passenger_col(data_2023)


/var/folders/km/dfm8dyn94yggw0mhf5zkplhr0000gn/T/ipykernel_72555/3950865421.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Passengers'] = df[['Passengers', 'Capacity']].min(axis=1)


In [59]:
data_2023_modelo.shape


(118953, 12)

In [62]:
data_2023_modelo.to_csv('data/data_2023_modelo.csv', index=False)


In [63]:
data_2024.to_csv('data/data_2024.csv', index=False)

In [64]:
data_2024

,Flight_ID,Aeronave,Capacity,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Passengers,Bookings,tipo_vuelo
1,0000a70ace2593137b79ace332478392,NaN,220,BN,AW,Ciudad Principal,MX Amigos y Familia,2024-06-13 20:00:00,2024-06-13 21:35:00,NaN,NaN,Corto
2,0000c5ba279c7225e9f6bac8490678e1,XA-VBT,220,AT,BD,Playa,Ciudad Principal,2024-03-03 15:10:00,2024-03-03 16:50:00,NaN,NaN,Corto
5,000131796f507492b61232912b171f28,NaN,186,BA,AK,Playa,Ciudad Principal,2024-10-11 10:30:00,2024-10-11 13:45:00,NaN,NaN,Largo
6,000158c9b43de5c11cb46d96b0501f2e,9H-AML,178,BN,AW,Ciudad Principal,MX Amigos y Familia,2024-04-23 17:45:00,2024-04-23 19:25:00,NaN,NaN,Corto
10,0003eae4dce252fd91f056612b615897,NaN,180,BL,AO,Ciudad Principal,Ecoturismo,2024-09-03 13:05:00,2024-09-03 15:05:00,NaN,NaN,Corto
...,...,...,...,...,...,...,...,...,...,...,...,...
238047,fffe7dce5952e95bdeecf51961bde767,NaN,180,AW,BH,Playa,Ciudad Principal,2024-07-06 08:20:00,2024-07-06 09:15:00,NaN,NaN,Corto
238048,fffec7d17cb550ce70c0d7fd8a203b4e,NaN,180,AT,AK,Playa,Ciudad Principal,2024-07-26 15:00:00,2024-07-26 18:25:00,NaN,NaN,Largo
238049,fffed485a13460817032edbb8eca9295,XA-VCC,230,AY,BA,Ciudad Principal,Playa,2024-04-05 14:20:00,2024-04-05 17:00:00,NaN,NaN,Largo
238051,ffff1e8f5485017f4c5a6a1919369156,XA-VIL,186,AT,AW,Ciudad Principal,Ciudad Principal,2024-04-23 05:40:00,2024-04-23 07:25:00,NaN,NaN,Corto


In [27]:

data_2023['STD'].dt.year.unique()

array([2023], dtype=int32)